# Workshop

Introduction to Langchain

In [5]:
# Load OpenAI key from env

import os
from dotenv import load_dotenv
load_dotenv(override=True)

azure_version = "2024-06-01"
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_embeddings = os.getenv("AZURE_OPENAI_EMBEDDINGS")
azure_whisper = os.getenv("AZURE_OPENAI_WHISPER")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_key = os.getenv("AZURE_OPENAI_KEY")

In [6]:
# create llm instance

from langchain_openai.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_deployment,
    azure_endpoint=azure_endpoint,
)

In [ ]:
# invoke llm

llm.invoke("What do you know about Conferences?")

In [ ]:
# invoke llm

answer = llm.invoke("What do you know about Conferences?")
print(answer.content)

In [ ]:
# use prompts

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""You are an encyclopedia.

Question: {input}""")

chain = prompt | llm

answer = chain.invoke({"input": "What are Conferences?"})
print(answer.content)

In [ ]:
# reusue prompts

answer = chain.invoke({"input": "What are Conferences also called?"})
print(answer.content)



In [ ]:
# contexts

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

chain = prompt | llm

answer = chain.invoke(
    {
        "input": "What are Conferences also called?",
        "context": "An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
    }
)
print(answer.content)

In [ ]:
# documents

from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain


documents = [
    Document(
        page_content="An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
        metadata={
            "source": "wikipedia"
        }
    )
]

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called?",
        "context": documents,
    }
)
print(answer)

In [ ]:
answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source?",
        "context": documents,
    }
)
print(answer)

In [ ]:
# document prompts

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}                             
Source: {source}""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt,
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source?",
        "context": documents,
    }
)
print(answer)

## Vector Stores

In [ ]:
# vector stores

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_chroma import Chroma


# load data
loader = TextLoader(r"./Data/Internal/budget.txt", encoding="utf-8")
data = loader.load()
loader = TextLoader(r"./Data/Internal/lostpassword.txt", encoding="utf-8")
data.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=30000, chunk_overlap=1000, separators=[".", "\n"])
documents = splitter.split_documents(data)

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_embeddings,
    azure_endpoint=azure_endpoint,
)
db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma/presentation")

for document in documents:
    print(document)
    print("------------------------")

In [ ]:
# retriever

from langchain.chains.retrieval import create_retrieval_chain

db = Chroma(persist_directory="./chroma/presentation", embedding_function=embeddings)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

answer = retrieval_chain.invoke({"input": "What is the title of step two when losing a password?"})
print(answer["answer"])

In [ ]:
answer = retrieval_chain.invoke({"input": "Give a concise description how I can apply for a budget."})
print(answer["answer"])

## Summary

In [ ]:
summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

Text: {context}""")

summary_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=summary_prompt,
    document_prompt=document_prompt,
)

answer = summary_chain.invoke(
    {"context": documents}
)
print(answer)

In [19]:
# from functools import partial

# from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
# from langchain.prompts import PromptTemplate
# from langchain.schema import StrOutputParser
# from langchain_core.prompts import format_document
# from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# partial_format_document = partial(format_document, prompt=document_prompt)

# # The chain we'll apply to each individual document.
# # Returns a summary of the document.
# map_chain = (
#     {"context": partial_format_document}
#     | summary_prompt
#     | llm
#     | StrOutputParser()
# )

# # A wrapper chain to keep the original Document metadata
# map_as_doc_chain = (
#     RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain})
#     | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
# ).with_config(run_name="Summarize (return doc)")

# # The chain we'll repeatedly apply to collapse subsets of the documents
# # into a consolidate document until the total token size of our
# # documents is below some max size.
# def format_docs(docs):
#     return "\n\n".join(partial_format_document(doc) for doc in docs)

# collapse_chain = (
#     {"context": format_docs}
#     | PromptTemplate.from_template("Collapse this content:\n\n{context}")
#     | llm
#     | StrOutputParser()
# )

# def get_num_tokens(docs):
#     return llm.get_num_tokens(format_docs(docs))

# def collapse(
#     docs,
#     config,
#     token_max=4000,
# ):
#     collapse_ct = 1
#     while get_num_tokens(docs) > token_max:
#         config["run_name"] = f"Collapse {collapse_ct}"
#         invoke = partial(collapse_chain.invoke, config=config)
#         split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
#         docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
#         collapse_ct += 1
#     return docs

# # The chain we'll use to combine our individual document summaries
# # (or summaries over subset of documents if we had to collapse the map results)
# # into a final summary.

# reduce_chain = (
#     {"context": format_docs}
#     | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
#     | llm
#     | StrOutputParser()
# ).with_config(run_name="Reduce")

# # The final full chain
# map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

# map_reduce.invoke(
#     input=documents,
#     config={"max_concurrency": 5},
# )

## Few shotting

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has. Only answer with two/three words.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.

Consider the following examples:

Question: Let's go to Tokyo next week.
Answer: Booking a flight

Question: Get some pizza delivered for this evening.
Answer: Ordering food

Question: How about a movie at 8pm at the cinema?
Anmswer: Getting a reservation

Now tell me which intent the follolwing question has. Only answer with two/three words.
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

## Agent and Tools

In [ ]:
answer = llm.invoke("What did Frank-Walter Steinmeier say in his christmas speech 2023?")
print(answer.content)

In [ ]:
from langchain.tools import Tool
from langchain.tools.ddg_search import DuckDuckGoSearchRun

ddg = DuckDuckGoSearchRun()

ddg_tool = Tool.from_function(
    func = ddg.run,
    name = "DuckDuckGo Search",
    description = "Search DuckDuckGo for a query about current events.",
)

tools = [ddg_tool]

ddg_tool.run("What did Frank-Walter Steinmeier say in his christmas speech 2023?")

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

In [ ]:
new_agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant and only answer in 10 words. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=new_agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)


agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

simple_summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand. Keep it funky!

Text: {input}""")
simple_summary_chain = {"input": RunnablePassthrough()} | simple_summary_prompt | llm

summary_tool = Tool(
    name="Funky Summary Tool",
    func=simple_summary_chain.invoke,
    description="Use this tool to do a funky summary. Make sure you get the text to do a summary of first."
)
tools.append(summary_tool)

agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?"})

In [ ]:
answer = llm.invoke("What is 1000 + 1234?")
print(answer.content)

In [ ]:
answer = llm.invoke("What is 13 raised to the .3432 power?")
print(answer.content)

## Images

In [ ]:
import base64
from mimetypes import guess_type
from langchain.prompts import HumanMessagePromptTemplate

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    # Default to png
    if mime_type is None:
        mime_type = 'image/png'

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


prompt_template =  HumanMessagePromptTemplate.from_template(
            template=[
                {"type": "text", "text": "Summarize this image"},
                {
                    "type": "image_url",
                    "image_url": "{encoded_image_url}",
                },
            ]
        )

summarize_image_prompt = ChatPromptTemplate.from_messages([prompt_template])

gpt4_image_chain = summarize_image_prompt | llm 

img_file = "Images/UserGuide.jpg"
page3_encoded = local_image_to_data_url(img_file)

answer = gpt4_image_chain.invoke(input={"encoded_image_url":page3_encoded})
print(answer.content)

## Audio

In [ ]:
import openai

client = openai.AzureOpenAI(
    api_key=azure_key,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_whisper,
    api_version=azure_version,
)

def get_transcript(audio_file):
    if not os.path.exists(audio_file):
        audio_file = "./Audio/" + audio_file
    client.audio.with_raw_response
    return client.audio.transcriptions.create(
        file=open(audio_file, "rb"),            
        model="whisper",
        language="de",
    ).text

audio_test_file = "./Audio/newyear2023.mp3"
print(get_transcript(audio_test_file))


In [ ]:
# audio
def get_audio_doc(input):
    return {
        "context": [Document(
            page_content=get_transcript(input),
            metadata={
                "source": "audio"
            }
        )]
    }
audio_summary_chain = get_audio_doc | summary_chain
answer = audio_summary_chain.invoke("./Audio/newyear2023.mp3")
print(answer)
